In [ ]:
%reset
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../")

import torch
import matplotlib.pyplot as plt

import implicitmodules.torch as dm

In [ ]:
dim = 2
sigma = 2.
sigma1 = 2.
nb_pts = 2

gd_trans = torch.tensor([[-2., 0.], [2., 0.]], requires_grad=True).view(-1)
mom_trans = torch.tensor([[0., -0.5], [0., 0.5]], requires_grad=True).view(-1)

gd_trans1 = torch.tensor([[-2., 0.], [2., 0.]], requires_grad=True).view(-1)
mom_trans1 = torch.tensor([[0., -0.5], [0., 0.5]], requires_grad=True).view(-1)

landmarks = dm.Manifolds.Landmarks(dim, nb_pts, gd=gd_trans, cotan=mom_trans)
landmarks1 = dm.Manifolds.Landmarks(dim, nb_pts, gd=gd_trans1, cotan=mom_trans1)

trans = dm.DeformationModules.Translations(landmarks, sigma)
trans1 = dm.DeformationModules.Translations(landmarks1, sigma1)
combination = dm.DeformationModules.CompoundModule([trans, trans1])
hamiltonian = dm.HamiltonianDynamic.Hamiltonian(combination)

In [ ]:
dm.HamiltonianDynamic.shoot_euler(hamiltonian, it=10)
print("Initial")
print("trans")
print(gd_trans.view(-1, 2))
print(mom_trans.view(-1, 2))
print("trans1")
print(gd_trans1.view(-1, 2))
print(mom_trans1.view(-1, 2))

print("Final")
print("trans")
print(trans.manifold.gd.view(-1, 2))
print(trans.manifold.cotan.view(-1, 2))
print(trans.controls.view(-1, 2))
print("trans1")
print(trans1.manifold.gd.view(-1, 2))
print(trans1.manifold.cotan.view(-1, 2))
print(trans1.controls.view(-1, 2))


In [ ]:
nx, ny = 100, 100
sx, sy = 10, 10
x, y = torch.meshgrid([torch.arange(0, nx), torch.arange(0, ny)])
x = sx*(x.type(torch.FloatTensor)/nx - 0.5)
y = sy*(y.type(torch.FloatTensor)/ny - 0.5)
u, v = dm.Utilities.vec2grid(trans(dm.Utilities.grid2vec(x, y).type(torch.FloatTensor)), nx, ny)

In [ ]:
%matplotlib qt5

plt.quiver(x.numpy(), y.numpy(), u.detach().numpy(), v.detach().numpy())
plt.show()